<a href="https://colab.research.google.com/github/Chandradithya8/Artificial_Neural_network/blob/main/Hyperparameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
df=pd.read_csv("/content/Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [13]:
a=pd.get_dummies(df[['Geography','Gender']],drop_first=True)
a.head()

,Geography_Germany,Geography_Spain,Gender_Male
0,0,0,0
1,0,1,0
2,0,0,0
3,0,0,0
4,0,1,0


In [14]:
df=df.drop(['RowNumber','CustomerId','Surname','Geography','Gender'],axis=1)
df=pd.concat([df,a],axis=1)
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0


In [15]:
x=df.drop('Exited',axis=1)
y=df['Exited']

In [16]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [17]:
from keras.models import Sequential
from keras.layers import Activation,Dense,Dropout,LeakyReLU,PReLU,ELU
from keras.activations import relu,sigmoid
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [18]:
def create_model(layers,activation):
  model=Sequential()
  for i,nodes in enumerate(layers):
    if i==0:
      model.add(Dense(units=nodes,input_dim=11))
      model.add(Activation(activation))
      model.add(Dropout(0.3))
    else:
      model.add(Dense(units=nodes))
      model.add(Activation(activation))
      model.add(Dropout(0.3))
  model.add(Dense(units=1,activation='sigmoid',kernel_initializer='he_uniform'))
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics='accuracy')
  return model   

In [19]:
model1=KerasClassifier(build_fn=create_model)

In [20]:
params={
    'layers':[(20), (40, 20), (45, 30, 15)], # if I use lists it shows me error
    'activation':['sigmoid','relu'],
    'batch_size':[128, 256], 
    'epochs':[30]
}
model2=GridSearchCV(estimator=model1,param_grid=params,cv=5)

In [21]:
result=model2.fit(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
TypeError: 'int' object is not iterable

  FitFailedWarning)


Epoch 1/30
50/50 [==============================] - 0s 1ms/step - loss: 0.5826 - accuracy: 0.7255
Epoch 2/30
50/50 [==============================] - 0s 1ms/step - loss: 0.5184 - accuracy: 0.7989
Epoch 3/30
50/50 [==============================] - 0s 1ms/step - loss: 0.5193 - accuracy: 0.7978
Epoch 4/30
50/50 [==============================] - 0s 1ms/step - loss: 0.5189 - accuracy: 0.7992
Epoch 5/30
50/50 [==============================] - 0s 1ms/step - loss: 0.5095 - accuracy: 0.7992
Epoch 6/30
50/50 [==============================] - 0s 1ms/step - loss: 0.5111 - accuracy: 0.7995
Epoch 7/30
50/50 [==============================] - 0s 1ms/step - loss: 0.5051 - accuracy: 0.8002
Epoch 8/30
50/50 [==============================] - 0s 2ms/step - loss: 0.5066 - accuracy: 0.7998
Epoch 9/30
50/50 [==============================] - 0s 1ms/step - loss: 0.5049 - accuracy: 0.8000
Epoch 10/30
50/50 [==============================] - 0s 1ms/step - loss: 0.5047 - accuracy: 0.7997
Epoch 11/30
50/50 [

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
TypeError: 'int' object is not iterable

  FitFailedWarning)


25/25 [==============================] - 0s 1ms/step - loss: 0.5522 - accuracy: 0.7575
Epoch 2/30
25/25 [==============================] - 0s 1ms/step - loss: 0.5204 - accuracy: 0.7942
Epoch 3/30
25/25 [==============================] - 0s 2ms/step - loss: 0.5182 - accuracy: 0.7981
Epoch 4/30
25/25 [==============================] - 0s 1ms/step - loss: 0.5198 - accuracy: 0.7966
Epoch 5/30
25/25 [==============================] - 0s 1ms/step - loss: 0.5144 - accuracy: 0.7972
Epoch 6/30
25/25 [==============================] - 0s 1ms/step - loss: 0.5117 - accuracy: 0.7969
Epoch 7/30
25/25 [==============================] - 0s 1ms/step - loss: 0.5124 - accuracy: 0.7987
Epoch 8/30
25/25 [==============================] - 0s 1ms/step - loss: 0.5064 - accuracy: 0.7984
Epoch 9/30
25/25 [==============================] - 0s 1ms/step - loss: 0.5095 - accuracy: 0.7997
Epoch 10/30
25/25 [==============================] - 0s 1ms/step - loss: 0.5076 - accuracy: 0.8002
Epoch 11/30
25/25 [===========

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
TypeError: 'int' object is not iterable

  FitFailedWarning)


50/50 [==============================] - 0s 1ms/step - loss: 4124.6577 - accuracy: 0.6139
Epoch 2/30
50/50 [==============================] - 0s 1ms/step - loss: 1625.5782 - accuracy: 0.6548
Epoch 3/30
50/50 [==============================] - 0s 1ms/step - loss: 686.4639 - accuracy: 0.6789
Epoch 4/30
50/50 [==============================] - 0s 1ms/step - loss: 299.5714 - accuracy: 0.7178
Epoch 5/30
50/50 [==============================] - 0s 1ms/step - loss: 163.4918 - accuracy: 0.7375
Epoch 6/30
50/50 [==============================] - 0s 1ms/step - loss: 87.1963 - accuracy: 0.7386
Epoch 7/30
50/50 [==============================] - 0s 2ms/step - loss: 67.9502 - accuracy: 0.7575
Epoch 8/30
50/50 [==============================] - 0s 1ms/step - loss: 46.5422 - accuracy: 0.7527
Epoch 9/30
50/50 [==============================] - 0s 1ms/step - loss: 36.9563 - accuracy: 0.7648
Epoch 10/30
50/50 [==============================] - 0s 2ms/step - loss: 24.1245 - accuracy: 0.7572
Epoch 11/30
5

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
TypeError: 'int' object is not iterable

  FitFailedWarning)


25/25 [==============================] - 0s 1ms/step - loss: 4599.2324 - accuracy: 0.6741
Epoch 2/30
25/25 [==============================] - 0s 2ms/step - loss: 2892.1519 - accuracy: 0.6559
Epoch 3/30
25/25 [==============================] - 0s 1ms/step - loss: 1905.6244 - accuracy: 0.6733
Epoch 4/30
25/25 [==============================] - 0s 1ms/step - loss: 1316.0405 - accuracy: 0.6602
Epoch 5/30
25/25 [==============================] - 0s 1ms/step - loss: 779.6301 - accuracy: 0.6673
Epoch 6/30
25/25 [==============================] - 0s 1ms/step - loss: 499.6648 - accuracy: 0.6698
Epoch 7/30
25/25 [==============================] - 0s 1ms/step - loss: 310.4344 - accuracy: 0.7014
Epoch 8/30
25/25 [==============================] - 0s 1ms/step - loss: 207.3500 - accuracy: 0.7220
Epoch 9/30
25/25 [==============================] - 0s 1ms/step - loss: 146.4088 - accuracy: 0.7355
Epoch 10/30
25/25 [==============================] - 0s 1ms/step - loss: 108.3862 - accuracy: 0.7425
Epoch 

In [22]:
result.best_score_

0.7953750014305114

In [23]:
result.best_params_

{'activation': 'sigmoid', 'batch_size': 128, 'epochs': 30, 'layers': (40, 20)}